# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies



Instructions for updating:
non-resource variables are not supported in the long term


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/contract-193200
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_

### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit North and hold:
hand = '5.AK93.AKJT72.T4'

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# the auction goes:
auction = ["PAD_START","PASS", "PASS", "2D"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, seat=0, dealer=1, ddsolver=dds, verbose=False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

# what's your bid?

Loaded lib dds.dll
DDSolver being loaded - dds mode 1


[{'call': '3D', 'insta_score': 0.896}]


In [3]:
bid.samples

[]

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START","PASS", "PASS", "2D"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: 0.7993
x.AK9x.AKJTxx.Tx QJx.Txx.9x.Q8xxx KT8xx.Qxx.x.KJ9x A9xx.J8x.Q8xx.Ax
x.AK9x.AKJTxx.Tx T8xx.Jxxx.Qx.KQx J9xx.QTx.8.A8xxx AKQx.8x.9xxx.J9x
x.AK9x.AKJTxx.Tx T9xx.QJxx..K8xxx KQxxx.T8x.Qx.J9x AJ8.xx.98xxx.AQx
x.AK9x.AKJTxx.Tx Axxx.T8xx.9.Qxxx J9x.QJxx.x.KJ98x KQT8x.x.Q8xxx.Ax
x.AK9x.AKJTxx.Tx K98xx.QT8x.8.Qxx QJxxx.Jxx.x.K98x AT.xx.Q9xxx.AJxx
x.AK9x.AKJTxx.Tx QJT9x.QT8xx.8.J9 K8xxx.xx.x.KQxxx Ax.Jx.Q9xxx.A8xx
x.AK9x.AKJTxx.Tx KJxx.T8xx.xx.K9x T8xxx.Qxx.9.AJxx AQ9.Jx.Q8xx.Q8xx
x.AK9x.AKJTxx.Tx QJT9x.T8xx.x.Axx Kxxx.QJx..KJ9xxx A8x.xx.Q98xxx.Q8
x.AK9x.AKJTxx.Tx A9xxx.Txx.8.AQxx QT8xx.QJxx..KJxx KJ.8x.Q9xxxx.98x
x.AK9x.AKJTxx.Tx QTxx.QJx.x.KJ9xx KJxx.Txxx.8x.Qxx A98x.8x.Q9xx.A8x
x.AK9x.AKJTxx.Tx KQT8x.Qxx.9.Qxxx Jxx.8xx.8x.KJ8xx A9xx.JTx.Qxxx.A9
x.AK9x.AKJTxx.Tx JTx.Qxx.x.AQ8xxx KQxxx.T8x.8x.Jxx A98x.Jxx.Q9xx.K9
x.AK9x.AKJTxx.Tx JTxx.QT8x..QJ9xx AQ9xx.Jxxx.xx.Kx K8x.x.Q98xx.A8xx
x.AK9x.AKJTxx.Tx 9xxx.QTx.x.AJ9xx AQT8x.xxx.x.Kxxx KJx.J8x.Q98xx.Q8
x.AK9x.AKJTxx.Tx Txx.Q8xx.x